In [156]:
import random
import itertools
import numpy as np

In [157]:
def generate_game(bigNumbers) -> tuple[list[int], int]:
    if bigNumbers > 4 or bigNumbers < 0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers = random.sample([25, 50, 75, 100], bigNumbers)
    smallNumbers = random.sample(
        list(range(1, 11)) + list(range(1, 11)), 6 - bigNumbers
    )

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))

In [158]:
operations = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}

In [159]:
def recusive_solve_countdown(numbers, target, calculations=[]):

    solutions = []

    # iterate over all pair of number pairs
    for pair in itertools.combinations(numbers, 2):

        # Remove the pair from the list of numbers and make copy
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        for op, calculate in operations.items():
            a = max(pair)
            b = min(pair)

            if (a == 0 or b == 0) or (a % b != 0) and op == "/":
                continue  # skip division by zero and none whole number division

            val = calculate(a, b)

            calcs = [*calculations, (a, op, b, val)]

            if val == target:
                solutions.append(calcs)

            sols = recusive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions = [*sols, *solutions]

    return solutions

In [160]:
def math_expression(expr, expresions):
    left = expresions.get(expr[0])

    if left is None:
        left = expr[0]
    else:
        del expresions[expr[0]]
        left = math_expression(left, expresions)

    right = expresions.get(expr[2])
    if right is None:
        right = expr[2]
    else:
        del expresions[expr[2]]
        right = math_expression(right, expresions)

    return f"({left} {expr[1]} {right})"

In [161]:
# game = generate_game(2)
game = ([75, 50, 2, 3, 7], 103)

print(game)
print()

solutions = recusive_solve_countdown(game[0], game[1])

print(len(solutions))
print()

for solution in solutions:
    for expr in solution:
        print(f"{expr[0]} {expr[1]} {expr[2]} = {expr[3]}")
    print()

    expressions = {}

    for expr in reversed(solution):
        expressions[expr[3]] = (expr[0], expr[1], expr[2])

    # recusivly expand the final expression
    print(math_expression(solution[-1], expressions))
    print()

([75, 50, 2, 3, 7], 103)

25

50 * 7 = 350
75 - 3 = 72
350 / 2 = 175
175 - 72 = 103

(((50 * 7) / 2) - (75 - 3))

50 * 7 = 350
350 / 2 = 175
75 - 3 = 72
175 - 72 = 103

(((50 * 7) / 2) - (75 - 3))

50 * 7 = 350
350 / 2 = 175
175 + 3 = 178
178 - 75 = 103

((((50 * 7) / 2) + 3) - 75)

50 * 7 = 350
350 / 2 = 175
175 - 75 = 100
100 + 3 = 103

((((50 * 7) / 2) - 75) + 3)

50 - 3 = 47
75 * 2 = 150
150 - 47 = 103

((75 * 2) - (50 - 3))

50 / 2 = 25
75 - 3 = 72
25 * 7 = 175
175 - 72 = 103

(((50 / 2) * 7) - (75 - 3))

50 / 2 = 25
75 + 3 = 78
78 + 25 = 103

((75 + 3) + (50 / 2))

50 / 2 = 25
25 * 7 = 175
75 - 3 = 72
175 - 72 = 103

(((50 / 2) * 7) - (75 - 3))

50 / 2 = 25
25 * 7 = 175
175 + 3 = 178
178 - 75 = 103

((((50 / 2) * 7) + 3) - 75)

50 / 2 = 25
25 * 7 = 175
175 - 75 = 100
100 + 3 = 103

((((50 / 2) * 7) - 75) + 3)

50 / 2 = 25
25 + 3 = 28
75 + 28 = 103

(75 + ((50 / 2) + 3))

50 / 2 = 25
75 + 25 = 100
100 + 3 = 103

((75 + (50 / 2)) + 3)

50 * 2 = 100
75 * 7 = 525
100 + 3 = 103

((50 